In [2]:
import cv2
import numpy as np
import time
from tkinter import *
from tkinter import filedialog 
import datetime

win=Tk()


def taking_file_name(files_name):
    
    def region_of_interest(img,vertices):
        mask=np.zeros_like(img)
        match_mask_color=255
        cv2.fillPoly(mask,vertices,match_mask_color)
        masked_image=cv2.bitwise_and(img,mask)
        return masked_image

    def draw_the_line(img,lines):
        img=np.copy(img)
        blank_image=np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
        if lines is not None:
            for line in lines:
                for x1,y1,x2,y2 in line:
                    cv2.line(blank_image,(x1,y1),(x2,y2),(0,255,0),thickness=3)
            img=cv2.addWeighted(img,0.8,blank_image,1,0.0)

        else:
            img=cv2.addWeighted(img,0.8,blank_image,1,0.0)
        return img
    def processing(img):
        height=img.shape[0]
        width=img.shape[1]
        region_of_interest_vertices=[
            (0,height),(width/2,height/2),(width,height)
        ]
        gray_image=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        canny_image=cv2.Canny(gray_image,100,200)

        cropped_image=region_of_interest(canny_image,
                                         np.array([region_of_interest_vertices],np.int32))

        lines=cv2.HoughLinesP(cropped_image,
                         rho=6,
                         theta=np.pi/60,
                         threshold=160,
                         lines=np.array([]),
                         minLineLength=50,
                         maxLineGap=10)
        image_with_lines=draw_the_line(img,lines)
        return image_with_lines
    cap=cv2.VideoCapture(files_name)
    c=0
    while cap.isOpened():
        ret,img=cap.read()
        c+=1
        font=cv2.FONT_HERSHEY_COMPLEX
        text='Press Esc to exit'
        time_text= str(datetime.datetime.now())
#         print(f'Type of time_text is {type(time_text)} ')
        if c%30:
            img=cv2.putText(img,text,(10,50),font,1.75,(0,0,255),2,
                        cv2.LINE_AA)
        img=cv2.putText(img,time_text,(550,50),font,1,(0,255,0),2,
                        cv2.LINE_AA)
        x=processing(img)
        cv2.imshow('frame',x)
        time.sleep(0.01)
        if cv2.waitKey(1) & 0xFF==27:
             break
    cap.release()
    cv2.destroyAllWindows()

def file_opener():
    filename = filedialog.askopenfilename(initialdir = "/", 
                    title = "Select a File", 
                    filetypes =[
                    ("all video format", ".mp4"),
                    ("all video format", ".flv"),
                    ("all video format", ".avi")])
    taking_file_name(filename)
def exit():
    win.destroy()
button=Button(text='Select the video',width=30,height=5,bg='green',fg='white',
       borderwidth=5,font=10,activebackground='red',command=file_opener).pack()
button=Button(text='Exit',width=30,height=5,bg='green',fg='white',
       borderwidth=5,font=10,activebackground='red',
              takefocus='blue',command=exit).pack()

win.config(bg='yellow')
win.geometry(f'{win.winfo_screenwidth()}x{win.winfo_screenheight()}')

win.mainloop()